**In this notebook, we will classify 15 thoracic findings from Chest X-ray images and associated reports. This can be considered as an VQA task. We will fine-tune 3 pre-trained transformer based V+L models. After running through this notebook, you will be able to fine-tune these models on your customized dataset.**

####**0.1 clone our repo and install dependencies!**


In [80]:
# !git clone https://github.com/YIKUAN8/Transformers-VQA.git
# %cd Transformers-VQA/
# !pip install -r requirements.txt


**Change the 79th line of param.py from**
```
args = parser.parse_args()
```
to
```
args = parser.parse_args([])
```
This will enable us to use *argparse* in jupyter notebook!



#### **0.2 Download pre-trained models and place them to data/pretrained/, you could choose from [VisualBERT](https://github.com/uclanlp/visualbert), [LXMERT](https://github.com/airsplay/lxmert), [UNITER](https://github.com/ChenRocks/UNITER).**

In [81]:
# #line 1: UNITER; line 2:LXMERT, line 3: VisualBERT. Comment out selected lines if you don't want to use this model
# #if the pre-trained VisualBERT cannot be downloaded succesfully, rerun one more time or refer to this link: https://drive.google.com/file/d/1kuPr187zWxSJbtCbVW87XzInXltM-i9Y/view?usp=sharing
# !wget https://convaisharables.blob.core.windows.net/uniter/pretrained/uniter-base.pt -P models/pretrained/
# !wget --no-check-certificate https://nlp1.cs.unc.edu/data/model_LXRT.pth -P models/pretrained/
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y" -O models/pretrained/visualbert.th && rm -rf /tmp/cookies.txt


#### **0.3 Download OpenI dataset.**

A detailed description of this dataset can be found at [here](https://openi.nlm.nih.gov/). In summary, there are 3684 CXR Image-Report pairs in this dataset. Each pair has an annotation of 15 throacic findings from MESH terms. We convert the raw data to a dataframe with better visibility. It can be accessed with the following command or this [link](https://drive.google.com/file/d/1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG/view?usp=sharing).

In [82]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG" -O data/openIdf.csv && rm -rf /tmp/cookies.txt

***0.3.1 Have a glance of this dataframe, column 'TXT' is the radiology report; column 'split' and 'id' are self-explantory; All other columns are the 15 findings. Our task will be a 15-labels binary classification with visual and semantic input.***

In [ ]:
import pandas as pd
import os

os.chdir('/home/andyalyfsyah/Transformers-VQA')

openI = pd.read_csv('data/openIdf.csv',index_col=0)
openI.shape
openI

In [ ]:
data_ct = pd.read_csv('https://raw.githubusercontent.com/AndiAlifs/Channelwise-Saab-Transform/main/data_ct.csv')
data_ct.head()

In [1]:
data_xray = pd.read_csv('https://raw.githubusercontent.com/AndiAlifs/Channelwise-Saab-Transform/main/data_xray.csv')

# set filename as index
data_xray.set_index('filename', inplace=True)
data_xray.drop(columns=['modality'], inplace=True)
# change clinical notes column name to TXT
data_xray.rename(columns={'clinical_notes':'TXT'}, inplace=True)

# add id with number from 1
data_xray['id'] = range(1, len(data_xray) + 1)

# add 'split' column where 20% contain 'test' and 80% contain 'train'
data_xray['split'] = 'train'
data_xray.loc[data_xray.sample(frac=0.2, random_state=42).index, 'split'] = 'test'

# move split column to second from last column
colname = ['id', 'todo', 'No Finding', 'Pneumonia', 'Pneumonia/Aspiration', 'Pneumonia/Bacterial', 'Pneumonia/Bacterial/Chlamydophila', 'Pneumonia/Bacterial/E.Coli', 'Pneumonia/Bacterial/Klebsiella', 'Pneumonia/Bacterial/Legionella', 'Pneumonia/Bacterial/Mycoplasma', 'Pneumonia/Bacterial/Nocardia', 'Pneumonia/Bacterial/Staphylococcus/MRSA', 'Pneumonia/Bacterial/Streptococcus', 'Pneumonia/Fungal/Aspergillosis', 'Pneumonia/Fungal/Pneumocystis', 'Pneumonia/Lipoid', 'Pneumonia/Viral/COVID-19', 'Pneumonia/Viral/Herpes ', 'Pneumonia/Viral/Influenza', 'Pneumonia/Viral/Influenza/H1N1', 'Pneumonia/Viral/MERS-CoV', 'Pneumonia/Viral/SARS', 'Pneumonia/Viral/Varicella', 'Tuberculosis', 'Unknown','split','TXT']
data_xray = data_xray[colname]

# change false to 0 and true to 1
data_xray = data_xray.replace(False, 0)
data_xray = data_xray.replace(True, 1)

# change to int
data_xray = data_xray.astype({'id': 'int32', 'todo': 'int32', 'No Finding': 'int32', 'Pneumonia': 'int32', 'Pneumonia/Aspiration': 'int32', 'Pneumonia/Bacterial': 'int32', 'Pneumonia/Bacterial/Chlamydophila': 'int32', 'Pneumonia/Bacterial/E.Coli': 'int32', 'Pneumonia/Bacterial/Klebsiella': 'int32', 'Pneumonia/Bacterial/Legionella': 'int32', 'Pneumonia/Bacterial/Mycoplasma': 'int32', 'Pneumonia/Bacterial/Nocardia': 'int32', 'Pneumonia/Bacterial/Staphylococcus/MRSA': 'int32', 'Pneumonia/Bacterial/Streptococcus': 'int32', 'Pneumonia/Fungal/Aspergillosis': 'int32', 'Pneumonia/Fungal/Pneumocystis': 'int32', 'Pneumonia/Lipoid': 'int32', 'Pneumonia/Viral/COVID-19': 'int32', 'Pneumonia/Viral/Herpes ': 'int32', 'Pneumonia/Viral/Influenza': 'int32', 'Pneumonia/Viral/Influenza/H1N1': 'int32', 'Pneumonia/Viral/MERS-CoV': 'int32', 'Pneumonia/Viral/SARS': 'int32', 'Pneumonia/Viral/Varicella': 'int32', 'Tuberculosis': 'int32', 'Unknown': 'int32'})

print(data_xray.shape)
data_xray

NameError: name 'pd' is not defined

In [6]:
data_ct = pd.read_csv('https://raw.githubusercontent.com/AndiAlifs/Channelwise-Saab-Transform/main/data_ct.csv')

# set filename as index
data_ct.set_index('filename', inplace=True)
data_ct.drop(columns=['modality'], inplace=True)
# change clinical notes column name to TXT
data_ct.rename(columns={'clinical_notes':'TXT'}, inplace=True)

# add id with number from 1
data_ct['id'] = range(867, 867+len(data_ct))

# add 'split' column where 20% contain 'test' and 80% contain 'train'
data_ct['split'] = 'train'
data_ct.loc[data_ct.sample(frac=0.2, random_state=42).index, 'split'] = 'test'

# move split column to second from last column
colname = ['id', 'todo', 'No Finding', 'Pneumonia', 'Pneumonia/Aspiration', 'Pneumonia/Bacterial', 'Pneumonia/Bacterial/Chlamydophila', 'Pneumonia/Bacterial/E.Coli', 'Pneumonia/Bacterial/Klebsiella', 'Pneumonia/Bacterial/Legionella', 'Pneumonia/Bacterial/Mycoplasma', 'Pneumonia/Bacterial/Nocardia', 'Pneumonia/Bacterial/Staphylococcus/MRSA', 'Pneumonia/Bacterial/Streptococcus', 'Pneumonia/Fungal/Aspergillosis', 'Pneumonia/Fungal/Pneumocystis', 'Pneumonia/Lipoid', 'Pneumonia/Viral/COVID-19', 'Pneumonia/Viral/Herpes ', 'Pneumonia/Viral/Influenza', 'Pneumonia/Viral/Influenza/H1N1', 'Pneumonia/Viral/MERS-CoV', 'Pneumonia/Viral/SARS', 'Pneumonia/Viral/Varicella', 'Tuberculosis', 'Unknown','split','TXT']
data_ct = data_ct[colname]

# change false to 0 and true to 1
data_ct = data_ct.replace(False, 0)
data_ct = data_ct.replace(True, 1)

# change to int
data_ct = data_ct.astype({'id': 'int32', 'todo': 'int32', 'No Finding': 'int32', 'Pneumonia': 'int32', 'Pneumonia/Aspiration': 'int32', 'Pneumonia/Bacterial': 'int32', 'Pneumonia/Bacterial/Chlamydophila': 'int32', 'Pneumonia/Bacterial/E.Coli': 'int32', 'Pneumonia/Bacterial/Klebsiella': 'int32', 'Pneumonia/Bacterial/Legionella': 'int32', 'Pneumonia/Bacterial/Mycoplasma': 'int32', 'Pneumonia/Bacterial/Nocardia': 'int32', 'Pneumonia/Bacterial/Staphylococcus/MRSA': 'int32', 'Pneumonia/Bacterial/Streptococcus': 'int32', 'Pneumonia/Fungal/Aspergillosis': 'int32', 'Pneumonia/Fungal/Pneumocystis': 'int32', 'Pneumonia/Lipoid': 'int32', 'Pneumonia/Viral/COVID-19': 'int32', 'Pneumonia/Viral/Herpes ': 'int32', 'Pneumonia/Viral/Influenza': 'int32', 'Pneumonia/Viral/Influenza/H1N1': 'int32', 'Pneumonia/Viral/MERS-CoV': 'int32', 'Pneumonia/Viral/SARS': 'int32', 'Pneumonia/Viral/Varicella': 'int32', 'Tuberculosis': 'int32', 'Unknown': 'int32'})


print(data_ct.shape)
data_ct.head()

(84, 28)


,id,todo,No Finding,Pneumonia,Pneumonia/Aspiration,Pneumonia/Bacterial,Pneumonia/Bacterial/Chlamydophila,Pneumonia/Bacterial/E.Coli,Pneumonia/Bacterial/Klebsiella,Pneumonia/Bacterial/Legionella,...,Pneumonia/Viral/Herpes,Pneumonia/Viral/Influenza,Pneumonia/Viral/Influenza/H1N1,Pneumonia/Viral/MERS-CoV,Pneumonia/Viral/SARS,Pneumonia/Viral/Varicella,Tuberculosis,Unknown,split,TXT
filename,,,,,,,,,,,,,,,,,,,,,
3ED3C0E1-4FE0-4238-8112-DDFF9E20B471.jpeg,867,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,test,"Standard CT, reconstruction with lung algorith..."
16745_7_4.png,868,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,Patient 2 - CT scan during hospitalization – a...
16745_4_2.png,869,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,Patient 3 - CT scan – axial view: bilateral an...
coronacases_org_009.nii.gz,870,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,CT images reveal ground-glass opacities on the...
16631_1_4.jpg,871,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,test,"A female patient, 39-years-old, fever (38.1℃) ..."


In [7]:
newlabel = [
    {
      "name": "No Finding",
      "conditions": [
        "No Finding"
      ]
    },
    {
      "name": "Pneumonia",
      "conditions": [
        "Pneumonia",
        "Pneumonia/Aspiration",
        "Pneumonia/Lipoid",
      ]
    },
    {
      "name": "Bacterial",
      "conditions": [
        "Pneumonia/Bacterial",
        "Pneumonia/Bacterial/Chlamydophila",
        "Pneumonia/Bacterial/E.Coli",
        "Pneumonia/Bacterial/Klebsiella",
        "Pneumonia/Bacterial/Legionella",
        "Pneumonia/Bacterial/Mycoplasma",
        "Pneumonia/Bacterial/Nocardia",
        "Pneumonia/Bacterial/Staphylococcus/MRSA",
        "Pneumonia/Bacterial/Streptococcus"
      ]
    },
    {
      "name": "Viral",
      "conditions": [
        "Pneumonia/Viral/COVID-19",
        "Pneumonia/Viral/Influenza",
        "Pneumonia/Viral/Influenza/H1N1",
        "Pneumonia/Viral/MERS-CoV",
        "Pneumonia/Viral/SARS",
        "Pneumonia/Viral/Varicella"
      ]
    },
    {
      "name": "Fungal",
      "conditions": [
        "Pneumonia/Fungal/Aspergillosis",
        "Pneumonia/Fungal/Pneumocystis"
      ]
    },
    {
      "name": "Unknown",
      "conditions": [
        "Tuberculosis",
        "Unknown",
        'todo'
      ]
    }
]

new_colname = ['id', 'No Finding', 'Pneumonia', 'Viral', 'Fungal' ,'Bacterial','Unknown' ,'split','TXT']


In [10]:
all_data = pd.concat([data_xray, data_ct])

print(all_data.shape)

(950, 28)


In [11]:
all_data_new = pd.DataFrame(columns=new_colname)

for i in range(len(all_data)):
    for j in range(len(newlabel)):
        if all_data.iloc[i][newlabel[j]['conditions']].sum() > 0:
            all_data_new = all_data_new.append(all_data.iloc[i])
            all_data_new.iloc[-1, all_data_new.columns.get_loc(newlabel[j]['name'])] = 1
            break

# drop all column that not in new_colname
all_data_new = all_data_new[new_colname]

all_data_new = all_data_new.fillna(0)

/tmp/ipykernel_300356/691120044.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data_new = all_data_new.append(all_data.iloc[i])
/tmp/ipykernel_300356/691120044.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data_new = all_data_new.append(all_data.iloc[i])
/tmp/ipykernel_300356/691120044.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data_new = all_data_new.append(all_data.iloc[i])
/tmp/ipykernel_300356/691120044.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data_new = all_data_new.append(all_data.iloc[i])
/tmp/ipykernel_300356/691120044.py:6: FutureWarning: The frame.append method is deprecated and w

In [12]:
all_data_new_train = all_data_new[all_data_new['split'] == 'train']
all_data_new_test = all_data_new[all_data_new['split'] == 'test']

In [13]:
# count all data label distribution
print(all_data_new_train.iloc[:,1:7].sum(axis=0))
print(all_data_new_train.shape)


No Finding     15
Pneumonia      77
Viral         504
Fungal         27
Bacterial      56
Unknown        78
dtype: int64
(757, 9)


In [14]:
print(all_data_new_test.iloc[:,1:7].sum(axis=0))
print(all_data_new_test.shape)

No Finding      7
Pneumonia      18
Viral         119
Fungal          5
Bacterial      17
Unknown        24
dtype: int64
(190, 9)


In [15]:
all_data_without_txt = pd.concat([data_xray, data_ct])
all_data_without_txt.TXT = 'none'

In [16]:
all_data_without_txt.drop(columns=['TXT'], inplace=True)

In [17]:
data_xray_without_txt = data_xray.copy()
data_xray_without_txt.TXT = 'none'

In [18]:
data_ct_without_txt = data_ct.copy()
data_ct_without_txt.TXT = 'none'

In [19]:
data_all_dict =  dict(zip(all_data.index, all_data.id))

In [20]:
# make a dictionary of data_xray key as key and data_xray id column as value
data_xray_dict = dict(zip(data_xray.index, data_xray.id))

In [21]:
data_ct_dict = dict(zip(data_ct.index, data_ct.id))

In [22]:
all_data_new_dict = dict(zip(all_data_new.index, all_data_new.id))

In [23]:
import pickle as pickle5

data_v_f = pickle5.load( open( "/home/andyalyfsyah/BERTHop/transformed.pickle", "rb" ))

In [24]:
# replicate data_v_f to new dictionary named data_v_f_new and change the key with data_xray_dict
data_v_f_new = dict()
for key, value in data_v_f.items():
    try:
        data_v_f_new[data_xray_dict[key]] = value
    except:
        pass


# get all name of file in data_v_f
data_v_f_name = list(data_v_f.keys())

# iterate over all_data index, if the index is not in data_v_f_name, drop the row
for index in all_data.index:
    if index not in data_v_f_name:
        all_data.drop(index=index, inplace=True)


len(data_v_f_new)


866

In [25]:
all_data_v_f = dict()
for key, value in data_v_f.items():
    try:
        # check if key is in data_all_dict
        if key in data_all_dict:
            # if yes, add to all_data_v_f dictionary
            all_data_v_f[data_all_dict[key]] = value
    except:
        pass
len(all_data_v_f)

929

In [26]:
data_ct_v_f = dict()
for key, value in data_v_f.items():
    try:
        # check if key is in data_ct_dict
        if key in data_ct_dict:
            # if yes, add to data_ct_v_f dictionary
            data_ct_v_f[data_ct_dict[key]] = value
    except:
        pass

In [27]:
all_data_new_v_f = dict()
for key, value in data_v_f.items():
    try:
        # check if key is in data_all_dict
        if key in all_data_new_dict:
            # if yes, add to all_data_v_f dictionary
            all_data_new_v_f[all_data_new_dict[key]] = value
    except:
        pass

all_data_new_v_f_name = list(all_data_new_v_f.keys())

for idData in all_data_new.id:
    if idData not in all_data_new_v_f_name:
        all_data_new.drop(all_data_new[all_data_new.id == idData].index, inplace=True)


print(all_data_new.shape)

(926, 9)


In [28]:
all_ct_keys = data_ct_v_f.keys()

# iterate over data_ct, if column id pf data_ct is not in all_ct_keys, drop the row
for index in data_ct.index:
    if data_ct.id[index] not in all_ct_keys:
        data_ct.drop(index=index, inplace=True)

In [29]:
all_data['TXT'].fillna('normal', inplace=True)
data_xray['TXT'].fillna('Normal', inplace=True)
data_ct['TXT'].fillna('Normal', inplace=True)

#### **0.4 Download the visaul features extracted by BUTD. 36 2048-dimension visual feature is extracted from each CXR Image. We use this [implementation](https://github.com/airsplay/py-bottom-up-attention). This step will take a while (~1min). To save downloading time, you can also make a copy of this [shareable link](https://drive.google.com/file/d/1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ/view?usp=sharing) to your own google drive and mount you colab to your gdrive.**


*If you are interested in the original CXR images, which is unnecessary to out project , you can access them [here](https://drive.google.com/drive/folders/1s5A0CFB6-2N5ThbuorUK1t-bUEKmZnjz?usp=sharing).*

In [16]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ" -O data/openI_v_features.pickle && rm -rf /tmp/cookies.txt

***0.4.1 Load visual features***

In [31]:
# get all Index Name
index_name = data_xray.index.values

xray_v_f = {}
for name in index_name:
    xray_v_f[name] = data_v_f[name]

In [32]:
all_data_temp = all_data_v_f

# create a new data_v_f_new 
all_data_v_f = dict()

# iterate over data_v_f_new_temp and for each iteration in dictionary add feature, bbox, img_w, img_h to data_v_f_new
for key, value in all_data_temp.items():
    # get width and height of image
    img_w, img_h = value.shape
    # get 12x12 matrix in the middle of value as feature
    feature = value[12:24, 12:24].reshape(36,4)
    # get 144 rows of value as bbox
    bbox = value[:144].reshape(36,2048)
    all_data_v_f[key] = (feature, bbox, (img_w, img_h))

In [33]:
# replicate data_v_f to new dictionary named data_v_f_new and change the key with data_xray_dict
data_v_f_new = dict()
for key, value in data_v_f.items():
    try:
        data_v_f_new[data_xray_dict[key]] = value
    except:
        pass

len(data_v_f_new)

# save data_v_f_new to a temporary variable
data_v_f_new_temp = data_v_f_new

# create a new data_v_f_new
data_v_f_new = dict()

# iterate over data_v_f_new_temp and for each iteration in dictionary add feature, bbox, img_w, img_h to data_v_f_new
for key, value in data_v_f_new_temp.items():
    # get width and height of image
    img_w, img_h = value.shape
    # get 12x12 matrix in the middle of value as feature
    feature = value[12:24, 12:24].reshape(36,4)
    # get 144 rows of value as bbox
    bbox = value[:144].reshape(36,2048)
    data_v_f_new[key] = (feature, bbox, (img_w, img_h))

In [34]:
data_ct_temp = data_ct_v_f

# create a new data_v_f_new 
data_ct_v_f = dict()

# iterate over data_v_f_new_temp and for each iteration in dictionary add feature, bbox, img_w, img_h to data_v_f_new
for key, value in data_ct_temp.items():
    # get width and height of image
    img_w, img_h = value.shape
    # get 12x12 matrix in the middle of value as feature
    feature = value[12:24, 12:24].reshape(36,4)
    # get 144 rows of value as bbox
    bbox = value[:144].reshape(36,2048)
    data_ct_v_f[key] = (feature, bbox, (img_w, img_h))

In [35]:
all_data_new_v_f_temp = all_data_new_v_f

# create a new data_v_f_new
all_data_new_v_f = dict()

# iterate over data_v_f_new_temp and for each iteration in dictionary add feature, bbox, img_w, img_h to data_v_f_new
for key, value in all_data_new_v_f_temp.items():
    # get width and height of image
    img_w, img_h = value.shape
    # get 12x12 matrix in the middle of value as feature
    feature = value[12:24, 12:24].reshape(36,4)
    # get 144 rows of value as bbox
    bbox = value[:144].reshape(36,2048)
    all_data_new_v_f[key] = (feature, bbox, (img_w, img_h))


In [36]:
print("Panjang semua data:", len(all_data))
print("panjang CT data:", len(data_ct))
print("panjang X-ray data:", len(data_xray))

Panjang semua data: 929
panjang CT data: 63
panjang X-ray data: 866


In [37]:
feature_example, bbox_example, (img_w_example, img_h_example) = all_data_v_f[all_data.id.iloc[5]]
feature_example.shape, bbox_example.shape, (img_w_example, img_h_example)

((36, 4), (36, 2048), (512, 512))

In [38]:
feature_example, bbox_example, (img_w_example, img_h_example) = data_v_f_new[data_xray.id.iloc[208]]
feature_example.shape, bbox_example.shape, (img_w_example, img_h_example)

((36, 4), (36, 2048), (512, 512))

In [39]:
feature_example, bbox_example, (img_w_example, img_h_example) = data_ct_v_f[data_ct.id.iloc[0]]
feature_example.shape, bbox_example.shape, (img_w_example, img_h_example)

((36, 4), (36, 2048), (512, 512))

In [40]:
feature_example, bbox_example, (img_w_example, img_h_example) = all_data_new_v_f[all_data_new.id.iloc[0]]
feature_example.shape, bbox_example.shape, (img_w_example, img_h_example)

((36, 4), (36, 2048), (512, 512))

In [41]:
assert len(list(xray_v_f.keys())) == len(data_xray.values), "Visual Features are inconsistent with openI dataset"

In [42]:
assert set(list(all_data_v_f.keys())) == set(all_data.id.values), "Visual Features are inconsistent with data_xray dataset"

In [43]:
assert set(list(data_ct_v_f.keys())) == set(data_ct.id.values), "Visual Features are inconsistent with data_ct dataset"

In [44]:
assert set(list(all_data_new_v_f.keys())) == set(all_data_new.id.values), "Visual Features are inconsistent with data_ct dataset"

#### Fixx the test dataset

In [45]:
# get data_ct where split is test
data_ct_test = data_ct[data_ct.split == 'test']
print("panjang data CT test:", len(data_ct_test))
print("panjang data CT train:", len(data_ct[data_ct.split == 'train']))

# get data_cxr where split is test
data_xray_test = data_xray[data_xray.split == 'test']
print("panjang data X-ray test:", len(data_xray_test))
print("panjang data X-ray train:", len(data_xray[data_xray.split == 'train']))

panjang data CT test: 15
panjang data CT train: 48
panjang data X-ray test: 173
panjang data X-ray train: 693


In [46]:
# print all_data with test split
all_data_test = data_ct_test.append(data_xray_test)
all_data_test.head()

/tmp/ipykernel_300356/4017633109.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data_test = data_ct_test.append(data_xray_test)


,id,todo,No Finding,Pneumonia,Pneumonia/Aspiration,Pneumonia/Bacterial,Pneumonia/Bacterial/Chlamydophila,Pneumonia/Bacterial/E.Coli,Pneumonia/Bacterial/Klebsiella,Pneumonia/Bacterial/Legionella,...,Pneumonia/Viral/Herpes,Pneumonia/Viral/Influenza,Pneumonia/Viral/Influenza/H1N1,Pneumonia/Viral/MERS-CoV,Pneumonia/Viral/SARS,Pneumonia/Viral/Varicella,Tuberculosis,Unknown,split,TXT
filename,,,,,,,,,,,,,,,,,,,,,
3ED3C0E1-4FE0-4238-8112-DDFF9E20B471.jpeg,867,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,test,"Standard CT, reconstruction with lung algorith..."
16631_1_4.jpg,871,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,test,"A female patient, 39-years-old, fever (38.1℃) ..."
16630_5_1.jpg,877,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,test,"A male patient, 22-years-old, fever (38.5C) an..."
16745_5_2.png,879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,test,Patient 4 - CT scan – axial view: bilateral ar...
16631_1_2.jpg,889,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,test,"A female patient, 39-years-old, fever (38.1℃) ..."


In [47]:
print("panjang all data test:", len(all_data_test))

panjang all data test: 188


In [48]:
all_data_test_v_f = dict()

# find all_data_test_v_f in data_v_f where key is equal to all_data_test.id
for key, value in data_v_f_new.items():
    if key in all_data_test.id.values:
        all_data_test_v_f[key] = value


In [49]:
# elminate if available in all_data_test but not in all_data_test_v_f
for key in all_data_test.id.values:
    if key not in all_data_test_v_f.keys():
        all_data_test = all_data_test[all_data_test.id != key]

        

In [50]:
assert set(list(all_data_test_v_f.keys())) == set(all_data_test.id.values), "Visual Features are inconsistent with all_data_test dataset"

In [51]:
data_ct_without_txt = data_ct_without_txt.TXT 

#### **Now We have download all data, models, and dependencies. We are good to go!!!**
**1. Change default arguments**

First, let's check it out!

In [52]:
# set working directory to the root of the project
import os

os.chdir('/home/andyalyfsyah/BERTHop')

from param import args

args.__dict__

{'model': 'lxmert',
 'train': 'train,nominival',
 'valid': 'minival',
 'test': None,
 'batch_size': 32,
 'optim': 'bert',
 'lr': 0.0001,
 'epochs': 2,
 'dropout': 0.1,
 'seed': 9595,
 'max_seq_length': 20,
 'output': 'models/trained/',
 'fast': False,
 'tiny': False,
 'tqdm': True,
 'load_trained': None,
 'load_pretrained': None,
 'from_scratch': False,
 'mce_loss': False,
 'multiGPU': False,
 'num_workers': 0,
 'optimizer': 'bert'}

***1.1*** Let's overwrite some arguments***

In [53]:
args.batch_size = 8
args.valid_batch_size = 8
args.epochs = 3
args.lr = 0.5
args.dropout = 0.1
args.model = 'visualbert' # use visualbert
# args.model = 'uniter' # use visualbert
args.load_pretrained = '/home/andyalyfsyah/Transformers-VQA/models/pretrained/visualbert.th' #load pretrained visualbert model
# args.load_pretrained = '/home/andyalyfsyah/Transformers-VQA/models/pretrained/uniter-base.pt' #load pretrained uniter-base model
args.max_seq_length = 128 #truncate or pad report lengths to 128 subwords

args.__dict__

{'model': 'visualbert',
 'train': 'train,nominival',
 'valid': 'minival',
 'test': None,
 'batch_size': 8,
 'optim': 'bert',
 'lr': 0.5,
 'epochs': 3,
 'dropout': 0.1,
 'seed': 9595,
 'max_seq_length': 128,
 'output': 'models/trained/',
 'fast': False,
 'tiny': False,
 'tqdm': True,
 'load_trained': None,
 'load_pretrained': '/home/andyalyfsyah/Transformers-VQA/models/pretrained/visualbert.th',
 'from_scratch': False,
 'mce_loss': False,
 'multiGPU': False,
 'num_workers': 0,
 'optimizer': 'bert',
 'valid_batch_size': 8}

#### **2. Create customized dataloader**

In [54]:
new_findings = list(all_data_new.columns[1:-2])
new_findings

['No Finding', 'Pneumonia', 'Viral', 'Fungal', 'Bacterial', 'Unknown']

In [55]:
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
class OpenIDataset(Dataset):
  def __init__(self, df, vf, split, model = 'lxmert'):
    # train_test_split and prepare labels
    self.dataset = df[df['split'] == split]
    self.visual_features = vf
    self.id_list = self.dataset.id.tolist()
    self.report_list = self.dataset.TXT.tolist()
    self.findings_list = self.dataset.columns[1:-2]
    self.target_list = self.dataset[self.findings_list].to_numpy().astype(np.float32)
    self.model = model

  def __len__(self):
    return len(self.id_list)

  def __getitem__(self, item):
    cxr_id = self.id_list[item]
    target = self.target_list[item]
    boxes, feats, (img_w, img_h) = self.visual_features[cxr_id]
    report = self.report_list[item]
    if self.model == 'uniter':
      boxes = self._uniterBoxes(boxes)
    if self.model == 'lxmert':
      boxes[:, (0, 2)] /= img_w
      boxes[:, (1, 3)] /= img_h
    return cxr_id, feats, boxes, report, target

  def _uniterBoxes(self, boxes):#uniter requires a 7-dimensiom beside the regular 4-d bbox
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes[:,1] = boxes[:,0]
    new_boxes[:,0] = boxes[:,1]
    new_boxes[:,3] = boxes[:,2]
    new_boxes[:,2] = boxes[:,3]
    new_boxes[:,4] = new_boxes[:,3]-new_boxes[:,1] #w
    new_boxes[:,5] = new_boxes[:,2]-new_boxes[:,0] #h
    new_boxes[:,6]=new_boxes[:,4]*new_boxes[:,5] #area
    return new_boxes  

In [56]:
# training = OpenIDataset(df = openI, vf = openI_v_f,  split='train', model = args.model)
# testing = OpenIDataset(df = openI, vf = openI_v_f,  split='test', model = args.model)
# training = OpenIDataset(df = all_data, vf = all_data_v_f,  split='train', model = args.model)
# testing = OpenIDataset(df = all_data, vf = all_data_v_f,  split='test', model = args.model)
# training = OpenIDataset(df = data_xray, vf = data_v_f_new,  split='train', model = args.model)
# testing = OpenIDataset(df = data_xray, vf = data_v_f_new,  split='test', model = args.model)
# training = OpenIDataset(df = data_ct, vf = data_ct_v_f,  split='train', model = args.model)
# testing = OpenIDataset(df = data_ct, vf = data_ct_v_f,  split='test', model = args.model)
# training = OpenIDataset(df = all_data_without_txt, vf = all_data_v_f,  split='train', model = args.model)
# training = OpenIDataset(df = data_xray_without_txt, vf = data_v_f_new,  split='train', model = args.model)
# training = OpenIDataset(df = data_ct, vf = data_ct_v_f,  split='train', model = args.model)
training = OpenIDataset(df = all_data_new, vf = all_data_new_v_f,  split='train', model = args.model)
testing = OpenIDataset(df = all_data_new, vf = all_data_new_v_f,  split='test', model = args.model)

# fixed_testing = OpenIDataset(df = all_data_test, vf = all_data_test_v_f,  split='test', model = args.model)

train_loader = DataLoader(training, batch_size=args.batch_size,shuffle=True, num_workers=0,drop_last=True, pin_memory=True)
test_loader = DataLoader(testing, batch_size=args.valid_batch_size,shuffle=False, num_workers=0,drop_last=False, pin_memory=True)
# test_loader = DataLoader(fixed_testing, batch_size=args.valid_batch_size,shuffle=False, num_workers=0,drop_last=False, pin_memory=True)

#### **3. Model, Optimizer, Loss Function, and Evaluation Function**

In [60]:
# chage working directory to the root of the project
os.chdir('/home/andyalyfsyah/Transformers-VQA')

from vqa_model import VQAModel

#init model
model = VQAModel(num_answers = len(new_findings), model = args.model)
# model = VQAModel(num_answers = len(findings), model = args.model)

In [ ]:
#load pretrained weights using VISUALBERT
model.encoder.load(args.load_pretrained)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
#send to GPU
model = model.cuda()


In [ ]:
import torch
loss = torch.nn.BCEWithLogitsLoss()

In [ ]:
from src.optimization import BertAdam

optim = BertAdam(list(model.parameters()),lr=args.lr,warmup=0.1,t_total=len(train_loader)*args.epochs)
# t_total denotes total training steps
# batch_per_epoch = len(train_loader)
# t_total = int(batch_per_epoch * args.epochs)

In [ ]:
# Evaluation function, we will report the AUC and accuray of each finding
def eval(target, pred):
    acc_list = []
    for i, d in enumerate(new_findings): #normal is excluded
    # for i, d in enumerate(findings[:-1]): #normal is excluded
        acc = np.mean(target[:,i] == (pred[:,i]>=0.5))
        print(i,d,acc)
        acc_list.append(acc)
    print('Averaged: '+str(np.average(acc_list)))

In [ ]:
# activation = torch.nn.Sigmoid()
activation = torch.nn.Softmax(dim=1)

#### **4. HIT and RUN**

### All Data New

In [59]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/92 [00:00<?, ?it/s]

NameError: name 'optim' is not defined

## Results

In [59]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/86 [00:00<?, ?it/s]

Epoch 0: Training Loss: tensor(6561.4746, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.9190751445086706
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.017341040462427744
16 Pneumonia/Viral/COVID-19 0.43352601156069365
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.9942196531791

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 1: Training Loss: tensor(888.5286, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.08092485549132948
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.9826589595375722
16 Pneumonia/Viral/COVID-19 0.43352601156069365
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.99421965317919

### UNITER

In [ ]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/92 [00:00<?, ?it/s]

Epoch 0: Training Loss: tensor(5002.3638, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/24 [00:00<?, ?it/s]

0 todo 0.898936170212766
1 No Finding 0.9627659574468085
2 Pneumonia 0.925531914893617
3 Pneumonia/Aspiration 0.9946808510638298
4 Pneumonia/Bacterial 0.9946808510638298
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9893617021276596
8 Pneumonia/Bacterial/Legionella 0.9840425531914894
9 Pneumonia/Bacterial/Mycoplasma 0.9680851063829787
10 Pneumonia/Bacterial/Nocardia 0.9946808510638298
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9946808510638298
12 Pneumonia/Bacterial/Streptococcus 0.9840425531914894
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.973404255319149
15 Pneumonia/Lipoid 0.9840425531914894
16 Pneumonia/Viral/COVID-19 0.5957446808510638
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9946808510638298
20 Pneumonia/Viral/MERS-CoV 0.9946808510638298
21 Pneumonia/Viral/SARS 0.9893617021276596
22 Pneumonia/Viral/Varicella 0.9946808510638298
23

  0%|          | 0/24 [00:00<?, ?it/s]

Epoch 1: Training Loss: tensor(750.3059, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/24 [00:00<?, ?it/s]

0 todo 0.898936170212766
1 No Finding 0.9627659574468085
2 Pneumonia 0.925531914893617
3 Pneumonia/Aspiration 0.9946808510638298
4 Pneumonia/Bacterial 0.9946808510638298
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9893617021276596
8 Pneumonia/Bacterial/Legionella 0.9840425531914894
9 Pneumonia/Bacterial/Mycoplasma 0.9680851063829787
10 Pneumonia/Bacterial/Nocardia 0.9946808510638298
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9946808510638298
12 Pneumonia/Bacterial/Streptococcus 0.9840425531914894
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.973404255319149
15 Pneumonia/Lipoid 0.9840425531914894
16 Pneumonia/Viral/COVID-19 0.5957446808510638
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9946808510638298
20 Pneumonia/Viral/MERS-CoV 0.9946808510638298
21 Pneumonia/Viral/SARS 0.9893617021276596
22 Pneumonia/Viral/Varicella 0.9946808510638298
23

## Only X-Ray

##### Using All_data_new

In [44]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))


  0%|          | 0/6 [00:00<?, ?it/s]

/home/andyalyfsyah/Transformers-VQA/src/optimization.py:142: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Epoch 0: Training Loss: tensor(63757.1641, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.9190751445086706
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.9826589595375722
16 Pneumonia/Viral/COVID-19 0.5664739884393064
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.9942196531791907

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 1: Training Loss: tensor(785.9106, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.9190751445086706
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.9826589595375722
16 Pneumonia/Viral/COVID-19 0.5664739884393064
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.9942196531791907

##### Using UNITER Architecture

In [ ]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/86 [00:00<?, ?it/s]

Epoch 0: Training Loss: tensor(7069.0850, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.9190751445086706
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.9826589595375722
16 Pneumonia/Viral/COVID-19 0.5664739884393064
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.9942196531791907

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 1: Training Loss: tensor(1380.0695, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.9190751445086706
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.9826589595375722
16 Pneumonia/Viral/COVID-19 0.5664739884393064
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.9942196531791907

#### Without TXT

##### All Data

In [135]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/92 [00:00<?, ?it/s]

AttributeError: 'OpenIDataset' object has no attribute 'report_list'

##### CXR Only

In [105]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/86 [00:00<?, ?it/s]

Epoch 0: Training Loss: tensor(7451.8032, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.9190751445086706
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.9826589595375722
16 Pneumonia/Viral/COVID-19 0.5664739884393064
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.9942196531791907

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 1: Training Loss: tensor(696.1970, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.9190751445086706
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.9826589595375722
16 Pneumonia/Viral/COVID-19 0.5664739884393064
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.9942196531791907

##### CT Only

In [107]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/6 [00:00<?, ?it/s]

Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.
Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.
Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.
Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.
Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.


Epoch 0: Training Loss: tensor(33.3823, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.9190751445086706
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.9826589595375722
16 Pneumonia/Viral/COVID-19 0.5664739884393064
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.9942196531791907

  0%|          | 0/22 [00:00<?, ?it/s]

Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.
Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.
Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.
Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.
Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.
Training beyond specified 't_total' steps with schedule 'warmup_linear'. Learning rate set to 0.0. Please set 't_total' of BertAdam correctly.


Epoch 1: Training Loss: tensor(33.3823, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/22 [00:00<?, ?it/s]

0 todo 0.8901734104046243
1 No Finding 0.9653179190751445
2 Pneumonia 0.9190751445086706
3 Pneumonia/Aspiration 0.9942196531791907
4 Pneumonia/Bacterial 0.9942196531791907
5 Pneumonia/Bacterial/Chlamydophila 1.0
6 Pneumonia/Bacterial/E.Coli 1.0
7 Pneumonia/Bacterial/Klebsiella 0.9884393063583815
8 Pneumonia/Bacterial/Legionella 0.9826589595375722
9 Pneumonia/Bacterial/Mycoplasma 0.9653179190751445
10 Pneumonia/Bacterial/Nocardia 0.9942196531791907
11 Pneumonia/Bacterial/Staphylococcus/MRSA 0.9942196531791907
12 Pneumonia/Bacterial/Streptococcus 0.9826589595375722
13 Pneumonia/Fungal/Aspergillosis 1.0
14 Pneumonia/Fungal/Pneumocystis 0.9710982658959537
15 Pneumonia/Lipoid 0.9826589595375722
16 Pneumonia/Viral/COVID-19 0.5664739884393064
17 Pneumonia/Viral/Herpes  1.0
18 Pneumonia/Viral/Influenza 1.0
19 Pneumonia/Viral/Influenza/H1N1 0.9942196531791907
20 Pneumonia/Viral/MERS-CoV 0.9942196531791907
21 Pneumonia/Viral/SARS 0.9884393063583815
22 Pneumonia/Viral/Varicella 0.9942196531791907